In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir(os.path.join('..', 'notebook_format'))
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from time import time
from joblib import cpu_count
from gensim.models import Word2Vec
from sklearn.datasets import fetch_20newsgroups

# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

%watermark -a 'Ethen' -d -t -v -p numpy,scipy,pandas,sklearn,tensorflow,gensim

Using TensorFlow backend.


Ethen 2017-07-31 20:15:12 

CPython 3.5.2
IPython 6.1.0

numpy 1.13.1
scipy 0.19.1
pandas 0.19.2
sklearn 0.18.1
tensorflow 1.2.1
gensim 2.3.0


# Word2vec (Skipgram)

At a high level `Word2Vec` is a unsupervised learning algorithm that uses a shallow neural network (with one hidden layer) to learn the vectorial representations of all the unique words/phrases for a given corpus. The advantage that word2vec offers is it tries to preserve the semantic meaning behind those terms. For example, a document may employ the words "dog" and "canine" to mean the same thing, but never use them together in a sentence. Ideally, the word2vec algorithm would be able to learn the context and place them together in similar vector semantic space.

We'll start off by using the Gensim's implementation of the algorithm to provide a high-level intuition.

In [3]:
newsgroups_train = fetch_20newsgroups(subset = 'train')

# gensim’s Word2vec expects a sequence of sentences as its input,
# where each sentence a list of words. We'll be lazy and not perform
# any sort of text preprocessing
documents = [doc.strip().split() for doc in newsgroups_train.data]

# exmaple output of the data
print('raw data:\n\n', newsgroups_train.data[0])
print('example input:\n', documents[0])

raw data:

 From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





example input:
 ['From:', 'lerxst@wam.umd.edu', "(where's", 'my', 'thing)', 'Subject:', 'WHAT', 'car', 'is', 'this!?', 'Nntp-Posting-Host:', 'rac3.wam.umd.edu', 'Organization:', 'University', 'of', 'Maryland,', 'College', 'Park', 'Lines:', '15', 'I', 'was', 'wonderi

In [4]:
# apart from the input sentence, the only additional paramter
# we'll set is to specify use all possible cpu to train the model
workers = cpu_count()
word2vec_checkpoint = 'word2vec'

start = time()
word2vec = Word2Vec(documents, workers = workers)
elapse = time() - start
print('elapse time:', elapse)

word2vec.save(word2vec_checkpoint)
word2vec = Word2Vec.load(word2vec_checkpoint)

# obtain the learned word vectors (.wv.syn0)
# and the vocabulary/word that corresponds to each word vector
word_vectors = pd.DataFrame(word2vec.wv.syn0, index = word2vec.wv.index2word)
print('word vector dimension: ', word_vectors.shape)
word_vectors.head()

elapse time: 9.132986068725586
word vector dimension:  (44593, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,0.960153,1.301804,-0.062724,-0.469118,0.673243,-1.453987,1.743939,0.655346,0.770092,-0.991302,...,0.284647,0.764507,-0.301485,-1.659431,-2.799000,-0.052547,2.272124,-0.366417,-0.399685,0.088958
to,1.629720,2.908779,0.608608,-2.245137,2.954733,-1.135437,2.811758,-1.996261,-0.630274,0.089899,...,-2.027514,-1.215050,-1.691848,-0.894485,0.075758,0.090979,2.994895,0.244743,-0.883106,0.651679
of,2.117018,1.140914,-2.116331,0.972621,-0.577451,-1.407348,3.242181,1.351841,0.002603,-1.864334,...,0.919263,-0.617555,2.502091,-1.333715,-0.829140,-0.207756,2.058630,0.442219,-0.973177,-0.238174
a,-1.238447,-2.529692,-0.274136,-1.020971,0.885126,-0.121562,3.244503,0.890045,-1.595537,0.298294,...,3.041430,1.563413,-2.133618,1.638698,-2.472653,0.584565,2.889527,-1.093240,1.765447,-0.544035
and,0.929484,-0.876599,0.963164,1.612499,0.575134,-0.022117,-0.441134,0.417722,1.509714,0.819744,...,0.777000,-0.218481,1.736312,-0.552094,-0.305773,-0.241215,0.734451,-0.761488,-0.574154,-1.121330


After the model has learned the word vectors, we can use them to look up related words and phrases (words that have similar semantic meaning) for a given term of interest by comparing distances between the vectors using distance metric such as cosine distance.

In [5]:
word2vec.wv.most_similar(positive = ['computer'], topn = 5)

[('keyboard', 0.8475836515426636),
 ('machine', 0.8429251909255981),
 ('network', 0.8330420255661011),
 ('application', 0.8271337747573853),
 ('modem', 0.8191824555397034)]

Apart from finding similar words using distance metric such as cosine distance, word vectors have the remarkable property that analogies between words seem to be encoded in the difference between word vectors. For example there seems to be a constant male-female difference vector:

<img src="img/word_vectors.png" width="40%" height="40%">

\begin{align}
W(woman) - W(man)
&\tilde{=} W(aunt) - W(uncle) \\
&\tilde{=} W(queen) - W(king)
\end{align}

This property means we also perform vector manipulation (e.g. addition and subtraction) with our word vectors. For example, you might have heard or saw the famous example of: King - male + female = queen.

## Model Details

The way the model works underneath the hood is that trains a neural network by do the following: Given a specific word in the middle of a sentence (the input word), look at the words nearby and pick one at random. 

There is a **window size** hyperparameter to the algorithm that quantifies the word "nearby". A typical window size might be 5, meaning 5 words behind and 5 words ahead (10 in total). There're some implementations that something even fancier: Instead of using a fix $k$ window around each word, the window is uniformly distributed from $1, 2, ..., K$, where $K$ is the max window size we specify.

The diagram below shows some of the training samples (word pairs) we would take from the sentence "The quick brown fox jumps over the lazy dog." We'll used a small window size of 2 just for the example. The word highlighted in blue is the input word.

<img src="img/skipgram.png" width="60%" height="60%">

After feeding a bunch of word pairs to the network, it is going to tell us the probability for every word in our vocabulary being "nearby" word that we chose. The output probabilities are going to relate to how likely it is find each vocabulary word nearby our input word. For example, if we gave the trained network the input word "Soviet", the output probabilities should be much higher for words like "Union" and "Russia" than for unrelated words like "watermelon" and "kangaroo".


So how is this all represented? First of all, we know we can't feed a word just as a text string to a neural network (or probably any machine learning model), i.e. we need a way to represent the words to the network. To do this, we first build a vocabulary of words from our training documents. We'll assume that our corpus has a vocabulary size of 10,000.

We’re going to represent an input word like "ants" as a one-hot vector. This vector will have 10,000 components (one for every unqiue word in our vocabulary) and we'll place a "1" in the position corresponding to the word "ants", and 0s in all of the other positions. The output of the network is a single vector (also with 10,000 components) containing, for every word in our vocabulary, the probability that a randomly selected nearby word is that vocabulary word. Here’s the architecture of our single-layer neural network.

<img src="img/word2vec_architecture.png" width="70%" height="70%">

There is no activation function on the hidden layer neurons, but the output neurons use softmax. We’ll come back to this later.

When training this network on word pairs, the input is a one-hot vector representing the input word and the training output is also a one-hot vector representing the output word. But when we evaluate the trained network on an input word, the output vector will actually be a probability distribution (i.e., a bunch of floating point values, not a one-hot vector).

An alternative diagram that depicts that Skip-gram model architecture well:

<img src="img/skipgram_architecture.png" width="30%" height="30%">

Where we're using the centre word $w_{(t)}$ to predict the surrounding words and the training objective is to learn word vector representations, a.k.a projections that are good at predicting the nearby words.

## The Hidden Layer

Let's say that we wish to learn word vectors with 300 features. The number of features is a hyperparameter that we would have to tune to our application to see which one yields the best result. So the hidden layer is going to be represented by a weight matrix with 10,000 rows (one for every word in our vocabulary) and 300 columns (one for every hidden neuron).

Now if we look at what would happen when we multiply the 1 x 10,000 one-hot vector representation of the word with a 10,000 x 300 matrix that represents the hidden layer's weight, it will effectively just select the matrix row corresponding to the "1". The following figure is a small example that does a matrix multiplication of a 1 x 5 one hot vector with a 5 x 2 hidden layer's weight to give you a visual. 

<img src="img/hidden_layer.png" width="70%" height="70%">

This means that the hidden layer of this model is really just operating as a lookup table. The output of the hidden layer is just the "word vector" for the input word.

## The Output Layer

The 1 x 300 word vector for "ants" then gets fed to the output layer. The output layer is a softmax regression classifier. There's another documentation on Softmax Regression [here](http://nbviewer.jupyter.org/github/ethen8181/machine-learning/blob/master/deep_learning/softmax.ipynb), but the gist of it is that each output neuron, one per word in our vocabulary will produce an output probability between 0 and 1 and the sum of all these output values will add up to 1.

Specifically, each output neuron has a weight vector which it multiplies against the word vector from the hidden layer, then it applies the function `exp(x)` to the result. Finally, in order to get the outputs to sum up to 1, we divide this result by the sum of the results from all 10,000 output nodes. Here’s an illustration of calculating the output probability for the word "car".

<img src="img/output_layer.png" width="70%" height="70%">

Note that neural network does not know anything about the offset of the output word relative to the input word. In other words, it does not learn a different set of probabilities for the word before the input versus the word after.

Recall that in the beginning of the documentation, we mentioned that the goal for word2vec is to represent each word in the corpus as the vector representation while trying to reserve semantic meaning. This means that if two different words have very similar "contexts" (that is, what words are likely to appear around them), then our model needs to output very similar results for these two words. And one way for the network to output similar context predictions for these two words is if the word vectors are similar. So to hit the notion home, if two words have similar contexts, then word2vec is motivated to learn similar word vectors for these two words!

# Improving Word2vec

You may have noticed that the skip-gram neural network contains a huge number of weights ... For our example with 300 features and a vocab of 10,000 words, that's 3M weights in the hidden layer and output layer each! Training this on a large dataset would be slow and prone to overfitting, so the word2vec authors introduced a number of tweaks to make training feasible.

- Modifying the optimization objective with a technique they called "Negative Sampling", which causes each training sample to update only a small percentage of the model’s weights
- Subsampling frequent words to decrease the number of training examples
- Treating common word pairs or phrases as single "words" in their model

It’s worth noting that subsampling frequent words and applying Negative Sampling not only reduced the compute burden of the training process, but also improved the quality of their resulting word vectors as well.


## Negative Sampling

Training a neural network means taking a training example and adjusting all of the neuron weights slightly so that it predicts that training sample more accurately. In other words, each training sample will tweak all of the weights in the neural network. As we discussed above, the size of our word vocabulary means that our skip-gram neural network has a tremendous number of weights, all of which would be updated slightly by every one of our billions of training samples! Negative sampling addresses this by having each training sample only modify a small percentage of the weights, rather than all of them. Here’s how it works:

When training the network on the word pair ("fox", "quick"), we want the "correct output" of the network, that is the output neuron corresponding to "quick" to output a 1, and for all of the other thousands of output neurons to output a 0.

With negative sampling, we are instead going to randomly select just a small number of "negative" words (let’s say 5) to update the weights for. (In this context, a "negative" word is one for which we want the network to output a 0 for). We will also still update the weights for our “positive” word (which is the word “quick” in our current example).

> The paper says that selecting 5-20 words works well for smaller datasets, and we can get away with only 2-5 words for large datasets.

Recall that the output layer of our model has a weight matrix that's 300 x 10,000. So we will just be updating the weights for our positive word ("quick"), plus the weights for 5 other words that we want to output 0. That’s a total of 6 output neurons, and 1,800 weight values total. That’s only 0.06% of the 3M weights in the output layer!

In the hidden layer, only the weights for the input word are updated (this is true whether you’re using Negative Sampling or not).

### Mathematical Notation

This section goes back and re-visit the mathematical notation for Word2vec skipgram's objective function. Hopefully, the math won't look so daunting after having an understanding of the model from a non-mathematical standpoint.

In this model we are given a corpus of words $w$ and their contexts $c$ (the word pair of the targeted word that we've sampled). We consider the conditional probabilities $p(c | w)$ and given a corpus $Text$, the goal is to set the parameters $\theta$ of $p(c | w; \theta)$ to maximize our objective function $J_\theta$, i.e. our the corpus probability, with regard to our model parameters $\theta$:

\begin{align}
J_{\theta} &= 
arg\underset{\theta}{max} \prod_{w \in Text} \bigg[ \prod_{c \in C(w)} p(c | w; \theta) \bigg]
\end{align}

Here $C(w)$ is word $w$'s set of contexts.

One approach for parameterizing the $p(c | w; \theta)$ part of the skip-gram model is the classic softmax objective function:

\begin{align}
p(c | w; \theta) &= \dfrac{exp(v_c \cdot v_w)}{ \sum_{c' \in C} exp(v_c' \cdot v_w)}
\end{align}

Where:

- $v_c$ and $v_w$ are the vector representations for word $c$ and $w$ respectively
- $C$ is the set of all available contexts

While the objective function above can be computed, it is computationally expensive to do due to the summation $\sum_{c' \in C} exp(v_c' \cdot v_w)$ since there can be thousands if not million of them. And this is where negative sampling comes in. 

Instead of trying to estimate the probability of the word pair directly, we train a model to differentiate the target word from noise (negative sample). We can thus reduce the problem of predicting the correct word to a binary classification task, where the model tries to distinguish positive/genuine data from noise/negative samples.

We know for the word pair we generated from the data we want to maximize its probability, i.e.

\begin{align}
arg\underset{\theta}{max} \prod_{(w, c) \in D} p(D = 1 \big| c, w; \theta)
\end{align}

Here:

- $D$ is the set of all word and context pairs we extract from the text
- $p(D = 1 \big| w, c)$ the probability that the word pair $(w, c)$ came from the corpus data

We then generate the set $D'$ of random (w, c) pairs, assuming they are all incorrect. The name "negative sampling" stems from the set $D'$ of randomly sampled negative examples. Note that when we pick one random word from the vocabulary, there is some tiny chance that the picked word is actually a valid context. If we consider the large number of vobulary we have, we can argue that the probability is really really tiny, but a lot of the packages still do take care of removing these "accidents".

Since this is a binary classification loss, we can use logistic regression to minimize the negative log-likelihood, leading to the objective function:

\begin{align}
J_{\theta} 
&= arg\underset{\theta}{max} 
\prod_{(w, c) \in D} p(D = 1 \big| c, w; \theta) \prod_{(w, c) \in D'} p(D = 0 \big| c, w; \theta) \\
&= arg\underset{\theta}{max} 
\prod_{(w, c) \in D} p(D = 1 \big| c, w; \theta) \prod_{(w, c) \in D'} \big(1 - p(D = 1 \big| c, w; \theta)\big) \\
&= arg\underset{\theta}{max} 
\sum_{(w, c) \in D} log \big(p(D = 1 \big| c, w; \theta)\big) \sum_{(w, c) \in D'} log \big(1 - p(D = 1 \big| c, w; \theta)\big) \\
&= arg\underset{\theta}{max} 
\sum_{(w, c) \in D} log \big( \dfrac{1}{1 + \text{exp}(-v_c \cdot v_w)} \big) \sum_{(w, c) \in D'} log \big (1 - \dfrac{1}{1 + \text{exp}(-v_c \cdot v_w)} \big) \\
&= arg\underset{\theta}{max}
\sum_{(w, c) \in D} log \big( \dfrac{1}{1 + \text{exp}(-v_c \cdot v_w)} \big) \sum_{(w, c) \in D'} log \big (\dfrac{1}{1 + \text{exp}(v_c \cdot v_w)} \big) \\
\end{align}

Note that there're various other approaches to approximate the expensive softmax objective function. Negative sampling is soley discussed here because it is fast and works really really well. If you would like to go deeper with this topic, the following link might be a good place to start. [Blog: On word embeddings - Part 2: Approximating the Softmax](http://ruder.io/word-embeddings-softmax/index.html)

### Selecting Negative Samples

One little detail that's missing from the description above is how do we select the negative samples.

The negative samples are chosen using the unigram distribution. Essentially, the probability of selecting a word as a negative sample is related to its frequency, with more frequenct words being more likely to be selected as negative samples. Instead of using the raw frequency for $w_i$, $\text{freq}(w_i)$, in the original word2vec paper, each word is given a weight that's equal to it's frequency (word count) raised to the 3/4 power. The probability for selecting a word is just it's weight divided by the sum of weights for all words.

\begin{align}
P(w_i) = \frac{ {\text{freq}(w_i)}^{3/4} }{\sum_{j=0}^{n} \left({\text{freq}(w_j)}^{3/4} \right) }
\end{align}

This decision to raise the frequency to the 3/4 power appears to be empirical; as the author claims it outperformed other functions (e.g. just using unigram distribution).

Side note: The way this selection is implemented in the original word2vec C code is interesting. They have a large array with 100M elements (which they refer to as the unigram table). They fill this table with the index of each word in the vocabulary multiple times, and the number of times a word’s index appears in the table is given by $P(w_i) \times \text{table_size}$. Then, to actually select a negative sample, we just generate a random integer between 0 and 100M, and use the word at that index in the table. Since the higher probability words occur more times in the table, we're more likely to pick those.

## Subsampling Frequenct Words

Word2vec has two additional parameters for discarding some of the input words: words appearing less
than `min-count` times are not considered as either words or contexts, and in addition frequent words are down-sampled as defined by the `sample` parameter.

There are two potential issues with frequently appeared words like "the":

- When looking at word pairs that includes "the", e.g. ("fox", "the"), "the" doesn’t tell us much about the meaning of "fox", since it appears in the context of pretty much every word.
- We will have more than enough samples of ("the", "the other word for the word pair") than we need to learn a good vector for "the".

Word2Vec implements a "subsampling" scheme to address this. For each word we encounter in our training text, there is a chance that we will discard it from the text. The probability that we cut the word is related to the word's frequency.

\begin{align}
\text{probability of keeping the word } w_i 
&= (\sqrt{\frac{z(w_i)}{0.001}} + 1) \cdot \frac{0.001}{z(w_i)}
\end{align}

Where:

- $z(w_i)$ is the fraction of the total words in the corpus that are that word. For example, if the word "peanut" occurs 1,000 times in a 1 billion word corpus, then z("peanut") = 1E-6.
- There is also a parameter called `sample` which controls how much subsampling occurs, and the default value is 0.001. Smaller values of `sample` mean words are less likely to be kept

Here are some interesting observations of this subsampling function (again this is using the default sample value of 0.001).

- $\text{probability of keeping the word } w_i = 1$ (100% chance of being kept) when $z(w_i) <= 0.0026$. This means that only words which represent more than 0.26% of the total words will be subsampled
- $\text{probability of keeping the word } w_i = 0.5$ (50% chance of being kept) when $z(w_i) <= 0.00746$
- $\text{probability of keeping the word } w_i = 0.033$ (3.3% chance of being kept) when $z(w_i) = 1.0$. That is, if the corpus consisted entirely of word $w_i$, which of course is ridiculous

## Detecting Phrases

Word pair like "Boston Globe" (a newspaper) has a much different meaning than the individual words "Boston" and "Globe". So it makes sense to treat "Boston Globe", wherever it occurs in the text, as a single word with its own word vector representation.

The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

\begin{align}
\frac{count(A B) - count_{min}}{count(A) \cdot count(B)} \cdot N > threshold
\end{align}

Where:

- $count(A)$ is the number of times token $A$ appears in the corpus
- $count(B)$ is the number of times token $B$ appears in the corpus
- $count(A B)$ is the number of times the tokens $A B$ appear in the corpus in this specific order
- $N$ is the total size of the corpus vocabulary
- $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times,
- $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase
    
As we can infer the formula is designed to make phrases out of words which occur together often relative to the number of individual occurrences. And a higher threshold value will favors phrases made of infrequent words in order to avoid making phrases out of common words like "and the" or "this is".

# Tensorflow Implementation

In [6]:
from subprocess import call
from zipfile import ZipFile


def read_data():
    """Read data into a list of tokens/words"""
    filename = 'text8.zip'
    base_url = 'http://mattmahoney.net/dc/'

    if not os.path.isfile(filename):
        call('wget ' + base_url + filename, shell = True)

    with ZipFile(filename) as f:
        file = f.namelist()[0]

        # ensure compatibility each python2 and python3's str type
        # https://stackoverflow.com/questions/37689802/what-is-tensorflow-compat-as-str
        data = tf.compat.as_str(f.read(file)).split()

    return data

In [7]:
# words = read_data()
# print('data:', words[:4])
# print('data size {}'.format(len(words)))

In [19]:
# each element is a document
newsgroups_train = fetch_20newsgroups(subset = 'train')
data = newsgroups_train.data

# gensim’s Word2vec expects a sequence of sentences as its input,
# where each sentence a list of words. We'll be lazy and not perform
# any sort of text preprocessing
documents = [doc.strip().split() for doc in data]

In [20]:
data[:2]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

words = [word.lower() 
         for doc in documents
         for word in doc
         if word not in ENGLISH_STOP_WORDS]
print('data:', words[:4])
print('data size {}'.format(len(words)))

data: ['from:', 'lerxst@wam.umd.edu', "(where's", 'thing)']
data size 2102819


In [10]:
from collections import Counter


def build_dataset(words, vocab_size = None):

    word_count = Counter(words).most_common(vocab_size)
    word_index = {word: idx for idx, (word, _) in enumerate(word_count)}

    # build up word index and replaced the words by its assigned indices
    data = []
    unknown_count = 0
    for word in words:
        if word in word_index:
            idx = word_index[word]
        else:
            idx = 0
            unknown_count += 1

        data.append(idx)

    # 'UNK' flag for out of vocabulary word
    unknown = 'UNK', unknown_count
    word_count.append(unknown)
    word_index_rev = {idx: word for word, idx in word_index.items()}
    return data, word_count, word_index, word_index_rev

In [11]:
# TODO : ??? do we need to return a 4 element tuple
data, word_count, word_index, word_index_rev = build_dataset(words)

print('Most common words', word_count[:5])
print('Sample data', data[:10])

Most common words [('i', 37616), ('>', 27843), ('the', 14994), ('subject:', 11644), ('from:', 11590)]
Sample data [4, 92817, 63207, 16505, 3, 45, 187, 91755, 21, 28772]


In [12]:
def generate_sample(indexed_words, window):
    """
    Form training pairs according to the skip-gram model
    
    Parameters
    ----------
    indexed_words : list
        list of index that represents the words, e.g. [5243, 3083, 11],
        and 5243 might represent the word "Today"
        
    window : int
        window size of the skip-gram model, where word is sampled before
        and after the center word according to this window size
    """
    for index, center in enumerate(indexed_words):
        # random integers from `low` (inclusive) to `high` (exclusive)
        context = np.random.randint(1, window + 1)

        # get a random target before the center word
        for target in indexed_words[max(0, index - context):index]:
            yield center, target

        # get a random target after the center word
        for target in indexed_words[(index + 1):(index + 1 + context)]:
            yield center, target

In [13]:
iterator = generate_sample(indexed_words = data, window = 3)

print('original data:', data[:6])
print('skip gram sample:')

# we start off by using the first word as the center word,
# and since there's no word before it, we will not have any
# sampled word before it; after that we keep sliding the center
# word and generate word pairs
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))

original data: [4, 92817, 63207, 16505, 3, 45]
skip gram sample:
(4, 92817)
(4, 63207)
(4, 16505)
(92817, 4)
(92817, 63207)
(63207, 92817)
(63207, 16505)
(16505, 63207)


In [14]:
def get_batch(iterator, batch_size):
    """
    Group a numerical stream of centered and targeted
    word into batches and yield them as numpy arrays
    """
    while True:
        center_batch = np.zeros(batch_size, dtype = np.int32)
        target_batch = np.zeros((batch_size, 1), dtype = np.int32)
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)

        yield center_batch, target_batch

In [15]:
window = 3
batch_size = 5
iterator = generate_sample(indexed_words = data, window = window)
batches = get_batch(iterator, batch_size)

# e.g. generate a batch
center_batch, target_batch = next(batches)
print(center_batch)
print(target_batch)

[    4     4 92817 92817 63207]
[[92817]
 [63207]
 [    4]
 [63207]
 [92817]]


**Step 1:** Define placeholders for input and output. Input is the center word and output is the target (context) word. Instead of using one-hot vectors, we input the index of those words directly.

```python
# explicitly naming our operations will make it easier later to track them
center_words = tf.placeholder(
    tf.int32, shape = [BATCH_SIZE], name = 'center_words')

# for target_words:
# we will use this with tensorflow's loss function later, and the function requires rank 2
# input, that's why there's an extra dimension in the shape
target_words = tf.placeholder(
    tf.int32, shape = [BATCH_SIZE, 1], name = 'target_words')
```

**Step 2:** Define the weight/variable. In this case, the embedding matrix. Each row corresponds to the representation vector of one word. If one word is represented with a vector of size EMBED_SIZE, then the embedding matrix will have shape [VOCAB_SIZE, EMBED_SIZE]. We initialize the embedding matrix to value from a random distribution. In this case, let’s choose uniform distribution.

```python
# word vectors
embed_matrix = tf.Variable(
    tf.random_uniform([VOCAB_SIZE, EMBED_SIZE], -1.0, 1.0), name = 'embed_matrix')
```

**Step 3:** Inference (compute the forward path of the graph). Recall that the hidden layer serves as a lookup table and its purpose is to get the vector representations of words in our dictionary.

<img src="img/hidden_layer.png" width="70%" height="70%">

i.e. The output of the hidden layer is just the "word vector" for the input word. Our embed_matrix has dimension [VOCAB_SIZE x EMBED_SIZE], with each row of the embedding matrix corresponds to the vector representation of the word at that index. So to get the representation of all the center words in the batch, we get the slice of all corresponding rows in the embedding matrix. TensorFlow provides a convenient method to do so called `tf.nn.embedding_lookup()`. This method is really useful when it comes to matrix multiplication with one-hot vectors because it saves us from doing a bunch of unnecessary computation that will return 0 anyway.

```python
# input -> hidden layer
embed = tf.nn.embedding_lookup(embed_matrix, center_words)
```

**Step 4: Define the loss function and optimizer** For nce_loss, we need weights and biases for the hidden layer to calculate negative sampling loss.

```python
# hidden layer -> output layer's weights
output_weight = tf.Variable(
    tf.truncated_normal([VOCAB_SIZE, EMBED_SIZE], stddev = 1.0 / EMBED_SIZE ** 0.5))

output_bias = tf.Variable(tf.zeros([VOCAB_SIZE]))

# hidden layer -> output layer + negative sampling loss
loss = tf.nn.sampled_softmax_loss(
    weights = output_weight, biases = output_bias,
    labels = target_words, inputs = embed,
    num_sampled = NUM_SAMPLED, num_classes = VOCAB_SIZE)

avg_loss = tf.reduce_mean(loss, name = 'loss')

# choose an optimizer to perform the heavy lifting
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
optimize = optimizer.minimize(avg_loss)
```

After defining the operations we will create the session to execute the computation, including feeding in the inputs, running the optimizer to minimize the objective function we just defined and fetch the loss value so we can check convergence. The following code chunk pretty much dumped everything into one giant function

In [16]:
hi

NameError: name 'hi' is not defined

In [ ]:
from tf_word2vec_no_frill import tf_word2vec


VOCAB_SIZE = len(word2vec.wv.vocab)  # 50000
BATCH_SIZE = 256
EMBED_SIZE = 100  # dimension of the word embedding vectors
WINDOW_SIZE = 5  # the context window 
NUM_SAMPLED = 10  # Number of negative examples to sample
LEARNING_RATE = 0.1
EPOCHS = 6
TENSORBOARD = './graphs/no_frills/'
word2vec_param = {'vocab_size': VOCAB_SIZE, 'batch_size': BATCH_SIZE,
                  'embed_size': EMBED_SIZE, 'num_sampled': NUM_SAMPLED,
                  'learning_rate': LEARNING_RATE, 'epochs': EPOCHS,
                  'tensorboard': TENSORBOARD, 'window_size': WINDOW_SIZE}

In [ ]:
# words = read_data()
words = [word.lower() 
         for doc in documents
         for word in doc
         if word not in ENGLISH_STOP_WORDS]
data, word_count, word_index, word_index_rev = build_dataset(words, VOCAB_SIZE)
# iterator = generate_sample(indexed_words = data, window = WINDOW_SIZE)
# batch_gen = get_batch(iterator, BATCH_SIZE)

# actual model training
# word_vectors, history = tf_word2vec(batch_gen, **word2vec_param)
word_vectors, history = tf_word2vec(data, **word2vec_param)

In [ ]:
# visualize the convergence or course
# we can also do this within tensorboard
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

plt.plot(history)
plt.title('Convergence Plot')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
from scipy.spatial.distance import cdist


top_k = 5
idx = word_index['computer']

eval_word = word_index_rev[idx]
print(eval_word, '\n')

# remember the cdist returns a the cosine distance,
# so when doing the argsort, which is sorting by ascending
# order, the top k most similar word will be the first k one;
# and since the most similar word will always be itself, we
# exclude that from the returned result
vector = word_vectors[idx].reshape(1, -1)
sim = cdist(word_vectors, vector, metric = 'cosine').ravel()
nearest_indices = np.argsort(sim)[1:(top_k + 1)]

for nearest_idx in nearest_indices:
    sim_word = word_index_rev[nearest_idx]
    print(sim_word)

In [ ]:
hi

In [ ]:
top = 500

embedding = word_vectors[:top]

metadata_file = 'top_vocab.tsv'
top_words = pd.DataFrame([word_index_rev[i] for i in range(top)])
top_words.to_csv(metadata_file, sep = '\t', index = False, header = False)
top_words.head()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from subprocess import call
from tensorflow.contrib.tensorboard.plugins import projector


def launch_tensorboard(embedding, log_dir, metadata_file = None):
    """
    Mainly use for visualizing embedding for now

    Parameters
    ----------
    embedding : str

    log_dir : str

    metadata_file : str

    References
    ----------
    Tensorflow Documentation: TensorBoard: Embedding Visualization
    - https://www.tensorflow.org/get_started/embedding_viz
    """
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)

    if isinstance(embedding, np.ndarray):
        embedding_var = tf.Variable(embedding, name = 'embedding')
    else:
        embedding = np.genfromtxt(embedding, dtype = np.float32, delimiter = '\t')
        embedding_var = tf.Variable(embedding, name = embedding_file.split('.')[0])

    # do a fake run of the model (not really running anything),
    # simply initialize the variable and save it to checkpoint
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    model_checkpoint = os.path.join(log_dir, 'model.ckpt')

    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess, model_checkpoint, global_step = 1)

    # boilerplate setup
    # we can add multiple embeddings, here we add only one
    config = projector.ProjectorConfig()
    config_embedding = config.embeddings.add()
    config_embedding.tensor_name = embedding_var.name

    if metadata_file is not None:
        # Link to its metadata file (e.g. labels that provides descriptions
        # for the each data points in the embeddings)
        config_embedding.metadata_path = metadata_file

    # use the same log_dir where we stored our model checkpoint
    summary_writer = tf.summary.FileWriter(log_dir)

    # the next line writes a projector_config.pbtxt in the log_dir
    # TensorBoard will read this file during startup
    projector.visualize_embeddings(summary_writer, config)

    # lauch TensorBoard
    call('tensorboard --logdir={}'.format(log_dir), shell = True)

In [ ]:
launch_tensorboard(embedding, log_dir = TENSORBOARD, metadata_file = metadata_file)

To terminate the tensorboard visualization in jupyter notebook we can go to the dropdown menu at the top: `Kernel -> Interrupt`

# Final Thoughts

## Hyperparameters

The most crucial decisions that affect the performance are the choice of the model
architecture, the size of the vectors, the subsampling rate, and the size of the training window

## Applications

Word2Vec and the concept of word embeddings originated in the domain of NLP, however the idea of words in the context of a sentence or a surrounding word window can be generalized to other problem domain dealing with sequences or sets of related data points. For example:

- A direct application of Word2Vec to a classical engineering task was recently presented by Spotify. They abstracted the ideas behind Word2Vec to apply them not simply to words in sentences but to any object in any sequence, in this case to songs in a playlist. Songs are treated as words and other songs in a playlist as their surrounding context, depending on whether the playlists in question were genre specific the vocabulary encompassed a number of songs from that collection. Now in order to recommend songs to a user one merely has to examine a neighborhood of the 'song embeddings' of songs the user already likes
- Similarly, we could recommend a user who to connect to in a social network setting by examining the graph of relationships, where the nodes represent words and a path through the graph represents a sentence, then nodes that occur in the context of similar other nodes, would be close together in the vector space.

These examples show that the general applicability of Word2Vec based algorithms is very rich and that it behooves practitioners to examine their problem domain with respect to sequences of objects that occur in some meaningful context.

## Resources

A lot of the non-mathematical notes were taken from Chris McCormick's blog post. If you wish to learn more about the topic, the following link also contains different resources that Chris has curated (including a commented version of word2vec's original C code, derivation of word2vec's gradient update, etc.). [Blog: Word2Vec Resources](http://mccormickml.com/2016/04/27/word2vec-resources/)

# Reference

- [Blog: Demystifying Word2Vec](http://www.deeplearningweekly.com/blog/demystifying-word2vec)
- [Blog: Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
- [Blog: Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
- [Note: CS 20SI Lecture note 4: How to structure your model in TensorFlow](http://web.stanford.edu/class/cs20si/lectures/notes_04.pdf)
- [Paper: Yoav Goldberg, Omer Levy (2014) word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)
- [Paper: Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, Jeffrey Dean (2013) Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)